# OCTIS Model Evaluation

## Prologue & Imports

We will evaluate the performance of most relevant OCTIS models as a baseline for non-SOTA Topic Modeling. These models will be compared on the same preprocessed dataset, the same number of topics and the same evaluation metrics.

In [1]:
from octis.models.LSI import LSI
from octis.models.NMF import NMF
from octis.models.LDA import LDA
from octis.models.HDP import HDP
from octis.models.NeuralLDA import NeuralLDA
from octis.models.ProdLDA import ProdLDA
from octis.dataset.dataset import Dataset
from octis.evaluation_metrics.coherence_metrics import Coherence
from octis.evaluation_metrics.diversity_metrics import TopicDiversity, KLDivergence
from octis.evaluation_metrics.similarity_metrics import RBO, PairwiseJaccardSimilarity
from octis.evaluation_metrics.topic_significance_metrics import KL_uniform

from spacy.lang.el.stop_words import STOP_WORDS as el_stop
from spacy.lang.en.stop_words import STOP_WORDS as en_stop

from utils.data_loader import GreekPMDataloader
from models.octis.utils.preprocessor_gr import GreekStanzaPreprocessor
from models.octis.config.preprocessing import preprocessor_gr_params
from models.octis.config.models import NUM_TOPICS, lsi_params, nmf_params, lda_params, hdp_params, neural_lda_params, prod_lda_params
from models.octis.config.optimization import OPTIMIZATION_RESULT_PATH, TOP_K, NUM_PROCESSES, MODEL_RUNS, search_space
from models.octis.utils.model_evaluator import OCTISModelEvaluator

import pandas as pd

2024-04-04 13:29:10 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-04-04 13:29:10 INFO: Downloaded file to /Users/dion/stanza_resources/resources.json
2024-04-04 13:29:10 WARNING: Language el package default expects mwt, which has been added
2024-04-04 13:29:11 INFO: Loading these models for language: el (Greek):
| Processor | Package                 |
---------------------------------------
| tokenize  | gdt                     |
| mwt       | gdt                     |
| pos       | models/oct..._tagger.pt |
| lemma     | models/oct...matizer.pt |

2024-04-04 13:29:11 INFO: Using device: cpu
2024-04-04 13:29:11 INFO: Loading: tokenize
2024-04-04 13:29:11 INFO: Loading: mwt
2024-04-04 13:29:11 INFO: Loading: pos
2024-04-04 13:29:11 INFO: Loading: lemma
2024-04-04 13:29:11 INFO: Done loading processors!


## Dataset Loading

If our dataset has already been processed and cached, then we can load it. Otherwise, we will preprocess it and save it for future use.

In [2]:
try:
    dataset = Dataset()
    dataset.load_custom_dataset_from_folder('models/octis/data/dataset')
    print("Dataset found cached - loading...")
except:
    print("Dataset not found in cache - loading...")
    # Merge data and prepare for preprocessing
    try:
        speeches_df = pd.read_csv('data/data_speeches.csv')
        statements_df = pd.read_csv('data/data_statements.csv')
    except: 
        print("GreekPM data not found - fetching...")
        ds = GreekPMDataloader() # If the data is not available, download it
        cats_df = ds.load_categories("speeches", "statements")
        print("GreekPM data fetched!")

    df = pd.concat([speeches_df, statements_df], ignore_index=True)
    
    # Drop irrelevant columns and convert to string
    df['text'] = df['text'].astype(str)
    df = df.drop(columns=['date', 'id', 'url', 'title']).dropna(how='any')
    
    df.to_csv('data/data_merged.csv', index=False)

    # We have some non-Greek stopwords in the dataset, so we need to remove them
    stopwords = set(el_stop).union(set(en_stop))
    
    # Initialize preprocessing
    preprocessor = GreekStanzaPreprocessor(
                             stopword_list=stopwords, 
                             **preprocessor_gr_params)
    
    # Create the dataset
    print("Preprocessing data...")
    dataset = preprocessor.preprocess_dataset(documents_path='data/data_merged.csv')
    
    dataset.save('models/octis/data/dataset/')
    print("Dataset preprocessed and saved!")

Dataset found cached - loading...


In [3]:
corpus = dataset.get_corpus()

## Evaluation Metrics

In [4]:
coherence_npmi = Coherence(texts=corpus, topk=TOP_K, processes=NUM_PROCESSES, measure='c_npmi')
coherence_cv = Coherence(texts=corpus, topk=TOP_K, processes=NUM_PROCESSES, measure='c_v')
coherence_umass = Coherence(texts=corpus, topk=TOP_K, processes=NUM_PROCESSES, measure='u_mass')
coherence_uci = Coherence(texts=corpus, topk=TOP_K, processes=NUM_PROCESSES, measure='c_uci')

diversity_topic = TopicDiversity(topk=TOP_K)
diversity_kl = KLDivergence()

similarity_rbo = RBO(topk=TOP_K)
similarity_pjs = PairwiseJaccardSimilarity()

significance_kluni = KL_uniform()

other_metrics = [coherence_npmi, coherence_umass, coherence_uci, diversity_topic, diversity_kl, similarity_rbo, similarity_pjs, significance_kluni]

In [5]:
metrics = {"coherence_npmi": coherence_npmi, "coherence_cv": coherence_cv, "coherence_umass": coherence_umass, "coherence_uci": coherence_uci, "diversity_topic": diversity_topic, "diversity_kl": diversity_kl, "similarity_rbo": similarity_rbo, "similarity_pjs": similarity_pjs, "significance_kluni": significance_kluni}

## Model Initialization

In [6]:
lsi_model = LSI(**lsi_params)
lda_model = LDA(**lda_params)
hdp_model = HDP(**hdp_params)
nmf_model = NMF(**nmf_params)
neural_lda_model = NeuralLDA(**neural_lda_params)
prod_lda_model = ProdLDA(**prod_lda_params)

In [7]:
models = {"lsi": lsi_model, "lda": lda_model, "hdp": hdp_model, "nmf": nmf_model, "neural_lda": neural_lda_model, "prod_lda": prod_lda_model}

## Evaluation

In [8]:
evaluator = OCTISModelEvaluator(dataset=dataset, 
                                models=models,
                                metrics=metrics,
                                topics=NUM_TOPICS,
                            )

In [9]:
evaluator.evaluate()

Epoch: [1/200]	Samples: [1439/287800]	Train Loss: 3378.429470335302	Time: 0:00:00.119144
Epoch: [1/200]	Samples: [160/32000]	Validation Loss: 3073.64375	Time: 0:00:00.005158
Epoch: [2/200]	Samples: [2878/287800]	Train Loss: 3304.1669887943017	Time: 0:00:00.107573
Epoch: [2/200]	Samples: [160/32000]	Validation Loss: 3075.122900390625	Time: 0:00:00.006745
Epoch: [3/200]	Samples: [4317/287800]	Train Loss: 3269.586035224114	Time: 0:00:00.106692
Epoch: [3/200]	Samples: [160/32000]	Validation Loss: 3068.27607421875	Time: 0:00:00.004176
Epoch: [4/200]	Samples: [5756/287800]	Train Loss: 3251.551696056289	Time: 0:00:00.101749
Epoch: [4/200]	Samples: [160/32000]	Validation Loss: 3058.075439453125	Time: 0:00:00.004851
Epoch: [5/200]	Samples: [7195/287800]	Train Loss: 3234.67709781098	Time: 0:00:00.116486
Epoch: [5/200]	Samples: [160/32000]	Validation Loss: 3039.773876953125	Time: 0:00:00.005505
Epoch: [6/200]	Samples: [8634/287800]	Train Loss: 3195.0554530055592	Time: 0:00:00.116149
Epoch: [6/200

/Users/dion/Library/CloudStorage/GoogleDrive-dion.rigatos@gmail.com/My Drive/Archivio/University/Classes/Erasmus Courses/NLP/NLP Project/greek-pm-topic-modeling/src/models/octis/utils/model_evaluator.py:47: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.evaluation_df = pd.concat([self.evaluation_df, pd.DataFrame(model_metric_data)], ignore_index=True)
/Users/dion/.pyenv/versions/3.11.8/envs/nlp-env/lib/python3.11/site-packages/octis/evaluation_metrics/diversity_metrics.py:244: RuntimeWarning: invalid value encountered in log
  divergence = np.sum(P*np.log(P/Q))
/Users/dion/.pyenv/versions/3.11.8/envs/nlp-env/lib/python3.11/site-packages/octis/evaluation_metrics/topic_significance_metrics.py:24: RuntimeWarning: invalid value encountered 

,model,coherence_npmi,coherence_cv,coherence_umass,coherence_uci,diversity_topic,diversity_kl,similarity_rbo,similarity_pjs,significance_kluni
0,lsi,0.124227,0.663027,-1.205847,0.609450,0.738462,0.885864,0.056632,0.034798,0.434231
1,lda,0.101787,0.656160,-1.101430,0.298030,0.907692,1.811897,0.016060,0.010660,1.310394
2,hdp,-0.051897,0.509259,-2.202071,-2.674711,0.554667,0.362610,0.016090,0.012933,0.214266
3,nmf,0.181904,0.699247,-1.021196,0.953869,0.723077,3.942570,0.054839,0.055419,2.125015
4,neural_lda,0.035713,0.619024,-1.528981,-1.075232,1.000000,0.978301,0.000000,0.000000,0.495215
5,prod_lda,-0.093289,0.559137,-1.808297,-4.493539,0.923077,NaN,0.009805,0.004049,NaN
